In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)  
AS
WITH sft AS (
    SELECT b.external_code po_doc, a.external_code sto_number
    FROM (
        SELECT external_code, po_code
        FROM {catalog_name}.udp_wcm_silver_realtime.supra_d_sto_header_spr
        WHERE client_code = 'WIN'
        QUALIFY ROW_NUMBER() OVER(PARTITION BY sto_code ORDER BY proc_date DESC) = 1
    ) a
    LEFT JOIN (
        SELECT po_code, external_code
        FROM {catalog_name}.udp_wcm_silver_realtime.d_po_header_spr
        QUALIFY ROW_NUMBER() OVER (PARTITION BY po_code ORDER BY proc_date DESC) = 1
    ) b
        USING(po_code)
    WHERE a.po_code IS NOT NULL
),
main AS (
    SELECT
        IFNULL(a.allocation_table, b.allocation_table) allocation_table, 
        IFNULL(a.allocation_item, b.allocation_item) allocation_item,
        'SAP' wms,
        a.po_doc, a.po_item, b.po_doc sto_number, b.po_item sto_item
    FROM (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table_sap WHERE po_doc LIKE '49%') a
    FULL JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table_sap WHERE po_doc LIKE '43%') b
        USING(allocation_table, allocation_item)
    UNION ALL
    SELECT
        NULL allocation_table, NULL allocation_item, 'SFT' wms,
        po_doc, NULL po_item, sto_number, NULL sto_item
    FROM sft
)
SELECT *
FROM main;
""")